对于 Pytorch 的神经网络来说，非常关键的一个库就是 `autograd` ，它主要是提供了对 Tensors 上所有运算操作的自动微分功能，也就是计算梯度的功能

它属于 `define-by-run` 类型框架，即反向传播操作的定义是根据代码的运行方式，因此每次迭代都可以是不同的

接下来会简单介绍一些例子来说明这个库的作用

### 张量

In [ ]:
import torch 
# 开始创建一个 tensor， 并设置 requires_grad=True 来追踪该变量相关的计算操作：
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [ ]:
# 执行任意计算操作，这里进行简单的加法运算：
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [ ]:
# y 是一个操作的结果，所以它带有属性 grad_fn：
print(y.grad_fn)

In [ ]:
# 继续对变量 y 进行操作：
z = y * y * 3
out = z.mean()
print('z=', z)
print('out=', out)

z= tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
out= tensor(27., grad_fn=<MeanBackward0>)


In [ ]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


### 梯度

In [ ]:
out.backward()
# 输出梯度 d(out)/dx
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [ ]:
x = torch.randn(3, requires_grad=True)

y = x * 2
# tensor1.data：返回一个和 tensor1 有着相同数据的 tensor，且返回的新 tensor 与 tensor1 共享同一个内存空间
# tensor.data.norm()：它对张量y每个元素进行平方，然后对它们求和，最后取平方根。 这些操作计算就是所谓的L2或欧几里德范数 
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 152.5833,  684.7393, -792.4349], grad_fn=<MulBackward0>)


In [ ]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


# ----------------------

### autograd.grad() 求导

In [19]:
import torch 
from torch.nn import functional as F
x = torch.ones(1)
print(x)
w = torch.full([1], 2, dtype=torch.float)
# 一定要在创建损失函数前将 w 的 requires_grad 属性设置为 True
# 表示该 tensor 对象需要梯度信息
w.requires_grad_()
print(w)
# 除了使用 tensor.requires_grad_() 方法将 tensor 的 requires_grad 属性改为 True 外
# 还可以在创建 tensor 时就设置 requires_grad 属性
b = torch.ones(1, requires_grad=True)
print(b)

# mse：均方差
# F.mse_loss(x*w, torch.ones(1)) 第一个参数是预测值（得分函数），第二个参数是 label（反过来其实也一样，因为要进行平方）
# 损失函数 loss = sum[(x*w-y)^2]
mse = F.mse_loss(x*w, torch.ones(1))
# 计算梯度
# 第一个参数是 loss 函数，第二个参数是个要被函数求偏导的所有自变量组成的列表
print(torch.autograd.grad(mse, [w]))


tensor([1.])
tensor([2.], requires_grad=True)
tensor([1.], requires_grad=True)
(tensor([2.]),)


### loss.backward() 求导

In [21]:
import torch 
from torch.nn import functional as F
x = torch.ones(1)
print(x)
w = torch.full([1], 2, dtype=torch.float)
# 将 tensor 的 requires_grad 属性设置为 True，表示开始追踪该变量上的所有操作
w.requires_grad_()
print(w)

# mse：均方差
# F.mse_loss(x*w, torch.ones(1)) 第一个参数是预测值（得分函数），第二个参数是 label（反过来其实也一样，因为要进行平方）
# 损失函数 loss = sum[(x*w-y)^2]
mse = F.mse_loss(x*w, torch.ones(1))
# 反向传播，自动计算梯度，没有返回值，梯度保存在属性 grad 中 
mse.backward()
print(w.grad)


tensor([1.])
tensor([2.], requires_grad=True)
tensor([2.])
